In [1]:
import pickle
with open('all_games_train.pkl', 'rb') as fp:
    all_games_train =  pickle.load(fp)
    
with open('all_games_test.pkl', 'rb') as fp:
    all_games_test =  pickle.load(fp)
    
with open('train_total.pkl', 'rb') as fp:
    train_total =  pickle.load(fp)
    
with open('test_total.pkl', 'rb') as fp:
    test_total =  pickle.load(fp)

In [2]:
import numpy as np
def advanced_representation(all_games, total):
    X = []
    y = []
    for team_name in all_games_train:
        for game in all_games_train[team_name]:
            opp_name = game[0][0]
            if opp_name == 'portland':
                opp_name = 'portland trail'
            if opp_name[-1] == '*':
                opp_name = opp_name[:-1].strip()
            year = game[2]
            label = int(game[1])
            past_games = game[0][1:-1]
            past_games = np.asarray(past_games)
            past_games = list(map(eval, past_games))
            location = list(game[0][-1])
            location = np.asarray(location)
            location = list(map(eval, location))
            active_index = [1,2,3,4,5,6,7,8,9,10,12,13,15,16,18,19,20,21,23]
            data1 = np.asarray(train_total[team_name][year])
            data1 = data1[active_index]
            data1 = list(map(eval, data1))
            data2 = np.asarray(train_total[opp_name][year])
            data2 = data2[active_index]
            data2 = list(map(eval, data2))
            data1 = np.asarray(data1)
            data2 = np.asarray(data2)
            diff = list((data1>data2)*1)
            feature = diff + past_games + location
            X.append(feature)
            y.append(label)
    return (np.array(X), np.array(y))

In [3]:
adv_train, adv_train_label = advanced_representation(all_games_train, train_total)
adv_test, adv_test_label = advanced_representation(all_games_test, test_total)

In [28]:
#Model 1 Perceptron
from sklearn.linear_model import Perceptron
def cal_accuracy(adv_train, adv_train_label, adv_test, adv_test_label):
    clf = Perceptron()
    clf.fit(adv_train, adv_train_label)
    y_pred = clf.predict(adv_test)
    return accuracy_score(adv_test_label, y_pred)

cal_accuracy(adv_train, adv_train_label, adv_test, adv_test_label)


In [29]:
# Model 2 SVM with kernel function
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
def cal_accuracy(adv_train, adv_train_label, adv_test, adv_test_label):
    svm = SVC(gamma = 'scale', kernel = 'poly')
    svm.fit(adv_train, adv_train_label)
    y_pred = svm.predict(adv_test)
    return accuracy_score(adv_test_label, y_pred)
cal_accuracy(adv_train, adv_train_label, adv_test, adv_test_label)

0.6957771935589878

In [32]:
adv_train.shape

(12172, 25)

In [79]:
#Model3 Fullly Connected Neural Network
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torchvision import transforms

class Dataset(Dataset):
    """CIFAR-10 image dataset."""
    def __init__(self, X, y, transformations=None):
        self.len = len(X)           
        self.x_data = torch.from_numpy(X).float()
        self.y_data = torch.from_numpy(y).long()
    
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]
    

class FeedForwardNN(nn.Module):
    """ 
        (1) Use self.fc1 as the variable name for your first fully connected layer
        (2) Use self.fc2 as the variable name for your second fully connected layer
    """
    def __init__(self):
        
        super(FeedForwardNN, self).__init__()
 
        self.fc1 = nn.Linear(25, 12)
        self.fc2 = nn.Linear(12, 2)


        

    def forward(self, x):
        out = F.tanh(self.fc1(x))
        out = F.sigmoid(self.fc2(out))

        
        return out
    

clf_nn = FeedForwardNN()



dataset = Dataset(adv_train, adv_train_label)

train_loader = DataLoader(dataset = dataset,
                          batch_size = 100,
                          shuffle = True)


dataset = Dataset(adv_test, adv_test_label)
test_loader = DataLoader(dataset = dataset,
                          batch_size = 100,
                        shuffle = True)

optimizer = optim.SGD(clf_nn.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 5e-4)
criterion = torch.nn.CrossEntropyLoss()

max_epochs = 1000

accuracy = np.zeros((max_epochs))

for epoch in range(max_epochs):
    for i, data in enumerate(train_loader, 0):

        # Get inputs and labels from data loader 
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)

        # Feed the input data into the network 
        y_pred = clf_nn(inputs)

        # Calculate the loss using predicted labels and ground truth labels

        loss = criterion(y_pred, labels)

        #print("epoch: ", epoch, "loss: ", loss.data[0])

        # zero gradient
        optimizer.zero_grad()

        # backpropogates to compute gradient
        loss.backward()

        # updates the weghts
        optimizer.step()

        # convert predicted laels into numpy
        y_pred_np = y_pred.data.numpy()

        # calculate the training accuracy of the current model

        label_np = labels.data.numpy().reshape(len(labels),1)

        correct = 0




correct = 0
total_size = 0
for i, data in enumerate(test_loader, 0):

    # Get inputs and labels from data loader 
    inputs, labels = data
    inputs, labels = Variable(inputs), Variable(labels)

    # Feed the input data into the network 
    y_pred = clf_nn(inputs)

    # convert predicted laels into numpy
    y_pred_np = y_pred.data.numpy()

    # calculate the training accuracy of the current model
    pred_np = y_pred_np  
    label_np = labels.data.numpy().reshape(len(labels),1)


    for j in range(y_pred_np.shape[0]):
        if np.argmax(pred_np[j,:]) == label_np[j,:]:
            correct += 1
    total_size += len(labels)
test_accuracy = float(correct) / float(total_size)







    


In [80]:
test_accuracy

0.6683371672691423